In [2]:
import mfem.ser as mfem
from bfp import *
from typing import Optional
import numpy as np


In [3]:
# Assembler Function
def assemble_system(fes, mu, w, mesh, S_arr, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff):
    marker = get_marker_for_mu(mesh, mu)
    v_coeff = VelocityCoefficient2(mu, S_arr)

    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -1.0), marker)
    b.Assemble()

    return A, b


# Solver Class with multiple solver options
class BoltzmannSolver:
    def __init__(self, solver_type=1, rel_tol=1e-12, abs_tol=1e-12, max_iter=500):
        self.solver_type = solver_type
        self.rel_tol = rel_tol
        self.abs_tol = abs_tol
        self.max_iter = max_iter

    def create_solver(self, A):
        if self.solver_type == 1:
            solver = mfem.GMRESSolver()
            prec = mfem.GSSmoother(A)
        elif self.solver_type == 2:
            solver = mfem.CGSolver()
            prec = mfem.DSmoother(A)
        elif self.solver_type == 3:
            solver = mfem.MINRESSolver()
            prec = mfem.GSSmoother(A)
        else:
            raise ValueError(f"Unknown solver type: {self.solver_type}")

        solver.SetOperator(A)
        solver.SetPreconditioner(prec)
        solver.SetRelTol(self.rel_tol)
        solver.SetAbsTol(self.abs_tol)
        solver.SetMaxIter(self.max_iter)
        solver.SetPrintLevel(0)

        return solver

    def solve_system(self, A, b, fes):
        solver = self.create_solver(A)
        psi = mfem.GridFunction(fes)
        psi.Assign(1.0)
        solver.Mult(b, psi)

        return psi, solver.GetNumIterations(), solver.GetFinalNorm()


# Example usage:
# A, b = assemble_system(fes, mu, w, mesh, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff)
# solver = BoltzmannSolver(solver_type=1)
# psi, iterations, final_norm = solver.solve_system(A, b, fes)
# psi.Save("solution.gf")


In [5]:
import mfem.ser as mfem

# Assembler Function
def assemble_system(fes, mu, mesh, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff):
    marker = get_marker_for_mu(mesh, mu)
    v_coeff = VelocityCoefficient2(mu, S_coeff)

    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -1.0), marker)
    b.Assemble()

    return A, b


# Solver Class with multiple solver options
class BoltzmannSolver:
    def __init__(self, solver_type=1, rel_tol=1e-12, abs_tol=1e-12, max_iter=500):
        self.solver_type = solver_type
        self.rel_tol = rel_tol
        self.abs_tol = abs_tol
        self.max_iter = max_iter

    def create_solver(self, A):
        if self.solver_type == 1:
            solver = mfem.GMRESSolver()
            prec = mfem.GSSmoother(A)
        elif self.solver_type == 2:
            solver = mfem.CGSolver()
            prec = mfem.DSmoother(A)
        elif self.solver_type == 3:
            solver = mfem.MINRESSolver()
            prec = mfem.GSSmoother(A)
        else:
            raise ValueError(f"Unknown solver type: {self.solver_type}")

        solver.SetOperator(A)
        solver.SetPreconditioner(prec)
        solver.SetRelTol(self.rel_tol)
        solver.SetAbsTol(self.abs_tol)
        solver.SetMaxIter(self.max_iter)
        solver.SetPrintLevel(0)

        return solver

    def solve_system(self, A, b, fes):
        solver = self.create_solver(A)
        psi = mfem.GridFunction(fes)
        psi.Assign(1.0)
        solver.Mult(b, psi)

        return psi, solver.GetNumIterations(), solver.GetFinalNorm()


# Solving loop function for multiple mu values
def solve_for_multiple_mu(fes, mu_vals, w_vals, mesh, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff, solver_type=1):
    solver = BoltzmannSolver(solver_type)
    psi_mu_list = []

    for mu, w in zip(mu_vals, w_vals):
        print("Solving for mu =", mu)
        A, b = assemble_system(fes, mu, mesh, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff)
        solution, iterations, final_norm = solver.solve_system(A, b, fes)
        print(f"  Solver iterations: {iterations}, final norm: {final_norm:.2e}")
        psi_mu_list.append((mu, w, solution))
        solution.Save(f"psi_mu_{mu:.3f}.gf")

    phi_new = mfem.GridFunction(fes)
    phi_new.Assign(0.0)

    for mu, w, psi in psi_mu_list:
        phi_new.Add(w, psi)

    phi_new.Save("phi_new.gf")
    
    return phi_new, psi_mu_list



In [6]:

# Set parameters
nx = 3
nE = 5
x_start = 0.0
x_end = 0.3
E_start = 1
E_end = 0.01
N_ang = 4
order = 1
mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
dim = mesh.Dimension()
#set_boundary_attribute(mesh, -1, x_start, E_start, x_end, tol=1e-8)
"""
vertex_array = mesh.GetVertexArray()
for i in range(mesh.GetNBE()):
    v_indices = mesh.GetBdrElementVertices(i)
    idx0 = v_indices[0]
    vx = vertex_array[idx0][0]
    vE = vertex_array[idx0][1]
    attr = mesh.GetBdrAttribute(i)
    print(f"Boundary element {i}: Vertex = ({vx}, {vE}), Attribute = {attr}")
"""
fec = mfem.DG_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetVSize()
print("Number of unknowns:", Size)
#ess_bdr = mfem.intArray(mesh.bdr_attributes.Max())
#ess_bdr.Assign(0) 
#ess_bdr[0] = 1 # Left boundary,  Dirichlet condition.
#ess_bdr[1] = 1 # Right boundary
mu_vals, w_vals = gauss_legendre_dirs(N_ang)
#E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(50)
S_arr = np.zeros(nE)
E_arr = np.linspace(E_start, E_end, nE+1)
xs_t_coeff = TotalXSCoefficient(5)
xs_s_coeff = ScatteringXSCoefficient(0)
S_coeff    = StoppingPowerCoefficient(0)
dS_dE_arr = np.zeros(nE)
dS_dE_coeff = StoppingPowerDerivativeCoefficient(0)
q_coeff = ConstantCoefficient(100)
inflow_coeff = ConstantCoefficient(0)
phi, psi_mu_list = solve_for_multiple_mu(fes, mu_vals, w_vals, mesh,
                                         S_arr, xs_t_coeff, dS_dE_coeff,
                                         q_coeff, inflow_coeff,
                                         solver_type=1)

: 

In [4]:
print(dS_dE_coeff.EvalValue([0, 0.5]))


NameError: name 'dS_dE_coeff' is not defined

In [5]:
def assemble_system(fes, mu, mesh, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff):
    mu_coeff = mfem.ConstantCoefficient(mu)
    marker = get_marker_for_mu(mesh, mu)
    v_coeff = VelocityCoefficient2(mu, S_coeff)

    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -1.0), marker)
    b.Assemble()

    return A, b